#### 선관위의 선거구-동 매칭 파일과 행정동 경계파일을 활용, 선거구 속성정보를 가진 shp 파일 생성

In [4]:
#중앙선관위 선거구내역 + 24.2 센서스 경계데이터
#부천시 및 아래의 내역(분동/합동) 미반영
#->선거구shp_22대총선_부천시가공
import pandas as pd
import os
import shutil
pd.set_option('display.max_rows', 10000)
import numpy as np
import geopandas as gpd

In [5]:
path_elctn = './22대총선_선거구_행정동'
filenames = os.listdir(path_elctn)

In [6]:
li_df = []
for i in filenames:    
    if i != '.DS_Store':
        df = pd.read_excel(f'{path_elctn}/{i}')
        df['시도명'] = i[:-5]
        df = df.iloc[3:,:].rename({'중앙선거관리위원회 선거통계시스템':'선거구명','Unnamed: 1':'정수'\
                                ,'Unnamed: 2':'구시군명','Unnamed: 3':'읍면동'}, axis = 1)
        df['선거구명'] = df['선거구명'].str[:-2]
        df['구시군명'] = df['구시군명'].str[:-2]
        li_df.append(df)
df_seongeo = pd.concat(li_df, axis = 0,ignore_index=True)
df_seongeo['정수'] = df_seongeo['정수'].apply(lambda x: 1 if x == '1  ' else 0 )


In [7]:
li_df = []
for i in range(len(df_seongeo)):
    df = df_seongeo.iloc[i,:]
    df_1 = pd.DataFrame(data = df['읍면동'].split(', '))
    df_1 = df_1.rename({0:'읍면동'}, axis = 1)
    li_df.append(df_1)

In [8]:
읍면동 = pd.concat(li_df, ignore_index= True, axis = 0)
#현 국회의원정수 : 300인(지역구 253 비례대표 47), 22대 총선의 투표정수 :
df_seongeo.정수.sum()
#선거구명 채우기, '  '를 결측치로 변환 후 ffill 필요
df_seongeo.loc[df_seongeo['정수'] == 0].선거구명.values
#'  ' 값 결측으로
df_seongeo['선거구명'] = df_seongeo['선거구명'].apply(lambda x : np.nan if x == '' else x)
df_seongeo['구시군명'] = df_seongeo['구시군명'].apply(lambda x: '세종시' if x == '세종특별자치시' else x)
# 선거구명 결측 앞행의 값으로 결측대체
df_seongeo['선거구명'] = df_seongeo['선거구명'].fillna(method = 'ffill')
df_seongeo['시도_선거구'] = df_seongeo['시도명']+ ' ' + df_seongeo['선거구명']
#하나로 뭉쳐있는 읍면동분해
#끝자리 공백 제거
df_seongeo['읍면동'] = df_seongeo['읍면동'].str[:-2]

/var/folders/_p/b25fmdzs0gs_7l54_j6744sm0000gn/T/ipykernel_16507/3298541415.py:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_seongeo['선거구명'] = df_seongeo['선거구명'].fillna(method = 'ffill')


In [9]:
li_df = [] # 데이터프레임을 집어넣을 리스트 생성
for i in range(len(df_seongeo)):
    df = df_seongeo.iloc[i,:] #선거구 1과 해당 선거구 내의 모든 행정동이 뭉쳐있는 객체를 데이터 프레임으로 전환 
    df_1 = pd.DataFrame(data = df['읍면동'].split(', ')) #-> 해당 선거구의 모든 행정동을 쪼갠 값을 data로 갖는 데이터프레임 생성
    #나머지 값은 그대로 넣기(행정동 수만큼 broad casting)
    df_1['선거구명'] = df['선거구명']
    df_1['정수'] = df['정수']
    df_1['구시군명'] = df['구시군명']
    df_1['시도명'] = df['시도명']
    df_1['시도_선거구'] = df['시도_선거구']
    df_1 = df_1.rename({0:'읍면동'}, axis = 1)#읍면동 컬럼에 값 부여
    li_df.append(df_1)#리스트에 데이터프레임 append

In [10]:
df_result = pd.concat(li_df, ignore_index= True, axis = 0)

In [11]:
#시도명 중 선거구분 등 불요 문자열 제거 -> 시도명
df_result['시도명'] = df_result['시도명'].str.split('[').str[2].str.strip(']')
#시도_선거구명 중 선거구분 등 불요 문자열 제거 -> 선거구명
df_result['시도_선거구'] = df_result['시도_선거구'].str.split(' ').str[1]

In [532]:
#읍면동 중 인구 조정으로 인해 실제 시군구 및 행정구와 다르게 조정이 되어있는 경우 
#괄호로 실제 시군구 및 행정구가 표기 되어있음 -> 분리하여 실제시군구 필드로 조정
#시군구명 또한 추후 shp과의 결합을 위해 추출 필요
df_result.loc[df_result['읍면동'].str.contains('\(')]
df_result['시군구_r'] = df_result['읍면동'].apply(lambda x: x.split('(')[1].strip(')') if '(' in x else '0')
#위에서 실제 시군구가 달라 골라낸 시군구조정 읍면동 외에는 다 기존의 시군구 
df_result.loc[df_result['시군구_r'] == '0', '시군구_r'] = df_result['구시군명']
df_result[['읍면동','시군구_r']].head()

,읍면동,시군구_r
0,목천읍,천안시동남구
1,북면,천안시동남구
2,성남면,천안시동남구
3,수신면,천안시동남구
4,병천면,천안시동남구


In [535]:
df_result['읍면동'] = df_result['읍면동'].apply(lambda x: x.split('(')[0] if '(' in x else x)
df_result.loc[df_result['구시군명'].str.contains('부천시')]

,읍면동,선거구명,정수,구시군명,시도명,시도_선거구,시군구_r
1677,성곡동,부천시갑,1,부천시오정구,경기도,부천시갑,부천시오정구
1678,원종1동,부천시갑,1,부천시오정구,경기도,부천시갑,부천시오정구
1679,원종2동,부천시갑,1,부천시오정구,경기도,부천시갑,부천시오정구
1680,고강본동,부천시갑,1,부천시오정구,경기도,부천시갑,부천시오정구
1681,고강1동,부천시갑,1,부천시오정구,경기도,부천시갑,부천시오정구
1682,오정동,부천시갑,1,부천시오정구,경기도,부천시갑,부천시오정구
1683,신흥동,부천시갑,1,부천시오정구,경기도,부천시갑,부천시오정구
1684,심곡1동,부천시갑,1,부천시오정구,경기도,부천시갑,부천시원미구
1685,심곡2동,부천시갑,1,부천시오정구,경기도,부천시갑,부천시원미구
1686,심곡3동,부천시갑,1,부천시오정구,경기도,부천시갑,부천시원미구


In [536]:
#중앙선관위 기준 선거구 -> 
len(df_result)

3553

### 아래부터는 레이어 파일 가공

In [13]:
file = gpd.read_file('./shp/행정경계_202402/시도_시군구_행정동/')
gdf = gpd.GeoDataFrame(file, crs = file.crs)
gdf.head()

,ADM_NM,ADM_CD,SIGUNGU_NM,SIDO_NM,SIGUNGU_CD,SIDO_CD,geometry
0,사직동,11010530,종로구,서울특별시,11010,11,"POLYGON ((197702.069 553187.311, 197703.431 55..."
1,삼청동,11010540,종로구,서울특별시,11010,11,"POLYGON ((198170.457 553770.678, 198172.189 55..."
2,부암동,11010550,종로구,서울특별시,11010,11,"POLYGON ((196621.023 556395.880, 196628.323 55..."
3,평창동,11010560,종로구,서울특별시,11010,11,"POLYGON ((197800.719 559064.245, 197782.581 55..."
4,한남동,11030740,용산구,서울특별시,11030,11,"POLYGON ((200404.268 550031.323, 200404.019 55..."


In [538]:
#선관위 기준 시도_시군구_행정동과 레이어의 값을 비교해서 서로 다른 개체를 찾아야함
#시도_시군구_행정동으로 키를 만들어 비교
df_result.columns = ['행정동','선거구','정수','구시군명','시도','시도_선거구','시군구_r']
gdf.columns = ['행정동','읍면동코드','시군구','시도','시군구코드','시도코드','geometry']
gdf['시도'] = gdf['시도'].apply(lambda x: '전북특별자치도' if x == '전라북도' else x)
df_result = df_result[['행정동','선거구','정수','구시군명','시도','시군구_r']]
df_result['시도_시군구_행정동'] = df_result['시도'] + '_' + df_result['시군구_r'] + '_' + df_result['행정동']
gdf['시도_시군구_행정동'] = gdf['시도'] + '_' + gdf['시군구'] + '_' + gdf['행정동']

In [539]:
display(df_result.head())
display(gdf.head())

,행정동,선거구,정수,구시군명,시도,시군구_r,시도_시군구_행정동
0,목천읍,천안시갑,1,천안시동남구,충청남도,천안시동남구,충청남도_천안시동남구_목천읍
1,북면,천안시갑,1,천안시동남구,충청남도,천안시동남구,충청남도_천안시동남구_북면
2,성남면,천안시갑,1,천안시동남구,충청남도,천안시동남구,충청남도_천안시동남구_성남면
3,수신면,천안시갑,1,천안시동남구,충청남도,천안시동남구,충청남도_천안시동남구_수신면
4,병천면,천안시갑,1,천안시동남구,충청남도,천안시동남구,충청남도_천안시동남구_병천면


,행정동,읍면동코드,시군구,시도,시군구코드,시도코드,geometry,시도_시군구_행정동
0,사직동,11010530,종로구,서울특별시,11010,11,"POLYGON ((197702.069 553187.311, 197703.431 55...",서울특별시_종로구_사직동
1,삼청동,11010540,종로구,서울특별시,11010,11,"POLYGON ((198170.457 553770.678, 198172.189 55...",서울특별시_종로구_삼청동
2,부암동,11010550,종로구,서울특별시,11010,11,"POLYGON ((196621.023 556395.880, 196628.323 55...",서울특별시_종로구_부암동
3,평창동,11010560,종로구,서울특별시,11010,11,"POLYGON ((197800.719 559064.245, 197782.581 55...",서울특별시_종로구_평창동
4,한남동,11030740,용산구,서울특별시,11030,11,"POLYGON ((200404.268 550031.323, 200404.019 55...",서울특별시_용산구_한남동


In [540]:
# 어차피 선거구 정보를 레이어에 넣어야함
# 레이어를 기준으로 선거구 정보를 비교
import unicodedata
import warnings
warnings.simplefilter("ignore")

In [541]:
# 선관위에서 다운 받은 자료는 자모가 분리되어 있음
print(len('충청남도_공주시_유구읍'))#선관위 다운 자료 copy -> 19 
print(len('충청남도_공주시_유구읍'))#내가 손으로 친거-> 12

19
12


In [542]:
# unicodedata 모듈을 이용한 형식 맞춰주기 -> df_result 전체 한글 데이터에 적용
print(len(unicodedata.normalize('NFC','충청남도_공주시_유구읍')))#선관위 다운 자료 copy -> 19 
print(len('충청남도_공주시_유구읍'))#내가 손으로 친거-> 12

12
12


In [543]:
df_result.loc[df_result['선거구'].str.contains('부천시')]

,행정동,선거구,정수,구시군명,시도,시군구_r,시도_시군구_행정동
1677,성곡동,부천시갑,1,부천시오정구,경기도,부천시오정구,경기도_부천시오정구_성곡동
1678,원종1동,부천시갑,1,부천시오정구,경기도,부천시오정구,경기도_부천시오정구_원종1동
1679,원종2동,부천시갑,1,부천시오정구,경기도,부천시오정구,경기도_부천시오정구_원종2동
1680,고강본동,부천시갑,1,부천시오정구,경기도,부천시오정구,경기도_부천시오정구_고강본동
1681,고강1동,부천시갑,1,부천시오정구,경기도,부천시오정구,경기도_부천시오정구_고강1동
1682,오정동,부천시갑,1,부천시오정구,경기도,부천시오정구,경기도_부천시오정구_오정동
1683,신흥동,부천시갑,1,부천시오정구,경기도,부천시오정구,경기도_부천시오정구_신흥동
1684,심곡1동,부천시갑,1,부천시오정구,경기도,부천시원미구,경기도_부천시원미구_심곡1동
1685,심곡2동,부천시갑,1,부천시오정구,경기도,부천시원미구,경기도_부천시원미구_심곡2동
1686,심곡3동,부천시갑,1,부천시오정구,경기도,부천시원미구,경기도_부천시원미구_심곡3동


In [544]:

li_col = list(df_result.columns)
li_col.remove('정수')
for i in li_col:
    print(i)
    for j in range(len(df_result)):
        df_result[i][j] = unicodedata.normalize('NFC',df_result[i][j])

행정동
선거구
구시군명
시도
시군구_r
시도_시군구_행정동


In [545]:
li_cor = list(gdf['시도_시군구_행정동'])
df_result['cor'] = df_result['시도_시군구_행정동'].\
    apply(lambda x : 1 if x in li_cor\
          else 0)

In [546]:
#불일치 값 1029개 대다수가 분동 결과 '제'가 붙은 경우
#레이어 파일에는 숫자앞에 '제'가 붙지 않음
#선관위 파일의 경우 '제'가 붙음
#-> 있는 것에서 없는 것으로 맞춰줌
len(df_result.loc[df_result['cor'] == 0])

821

In [547]:
#check_point
#아래는 '제'를 골라내는 코드
df_result['분동여부']=df_result.loc[df_result['행정동'].str.contains('제')]['행정동'].str.split('제').str[1].str[0]
print(len(df_result))
df_result_1 = df_result.loc[df_result['분동여부'].isin(['1','2','3','4','5','6','7','8','9'])]
df_result_2 = df_result.loc[~df_result['분동여부'].isin(['1','2','3','4','5','6','7','8','9'])]
print(len(df_result_1))
print(len(df_result_2))
df_result_1['행정동'] = df_result_1['행정동'].str.replace('제','')
df_result_1['행정동'] = df_result_1['행정동'].apply(lambda x: x.replace('제제','제') if '제제' in x else x)
df_result = pd.concat([df_result_1,df_result_2], axis = 0).drop('분동여부', axis = 1)
df_result['구시군명'] = df_result['구시군명'].apply(lambda x:'화성시' if '화성시' in x else x)
df_result['행정동'] = df_result['행정동'].apply(lambda x:x.split('(')[0])
df_result['cor'] = df_result['구시군명'] + df_result['행정동']

3553
326
3227


In [548]:
#'제'에 대한 처리 후의 결과
df_result['시도_시군구_행정동'] = df_result['시도'] + '_' + df_result['시군구_r'] + '_' + df_result['행정동']
li_cor = list(gdf['시도_시군구_행정동'])
df_result['cor'] = df_result['시도_시군구_행정동'].\
    apply(lambda x : 1 if x in li_cor\
          else 0)

In [549]:
#'제' 처리 후에는 717건으로 보정
len(df_result.loc[df_result['cor'] == 0])

509

In [550]:
# '구'가 있는 시의 경우 
gdf['행정구여부'] = gdf['시군구'].apply(lambda x: x.split(' ')[0]\
                                 if len(x.split(' ')) == 2 else 0)
li_gu = list(gdf['행정구여부'].unique())
li_gu.remove(0)
print(len(li_gu))
print(li_gu)

12
['포항시', '고양시', '창원시', '천안시', '청주시', '안양시', '수원시', '성남시', '부천시', '안산시', '용인시', '전주시']


In [551]:
# li_gu 구가 있는 기초지자체를 이용, 해당 기초지자체와 구를 ' '로 분리
df_result.loc[df_result['선거구'].str.contains('부천')].head()

,행정동,선거구,정수,구시군명,시도,시군구_r,시도_시군구_행정동,cor
1677,성곡동,부천시갑,1,부천시오정구,경기도,부천시오정구,경기도_부천시오정구_성곡동,0
1678,원종1동,부천시갑,1,부천시오정구,경기도,부천시오정구,경기도_부천시오정구_원종1동,0
1679,원종2동,부천시갑,1,부천시오정구,경기도,부천시오정구,경기도_부천시오정구_원종2동,0
1680,고강본동,부천시갑,1,부천시오정구,경기도,부천시오정구,경기도_부천시오정구_고강본동,0
1681,고강1동,부천시갑,1,부천시오정구,경기도,부천시오정구,경기도_부천시오정구_고강1동,0


In [552]:
li_df = []
for i in li_gu:
    df = df_result.loc[df_result['시군구_r'].str.contains(i)]
    li_df.append(df)
    print(i,len(df))
print(len(li_df))

포항시 29
고양시 44
창원시 55
천안시 31
청주시 43
안양시 31
수원시 44
성남시 50
부천시 37
안산시 25
용인시 38
전주시 35
12


In [553]:
#행정구를 가진 경우
df_result_1 = pd.concat(li_df, axis = 0)
df_result_1.head()

,행정동,선거구,정수,구시군명,시도,시군구_r,시도_시군구_행정동,cor
2683,흥해읍,포항시북구,1,포항시북구,경상북도,포항시북구,경상북도_포항시북구_흥해읍,0
2684,신광면,포항시북구,1,포항시북구,경상북도,포항시북구,경상북도_포항시북구_신광면,0
2685,청하면,포항시북구,1,포항시북구,경상북도,포항시북구,경상북도_포항시북구_청하면,0
2686,송라면,포항시북구,1,포항시북구,경상북도,포항시북구,경상북도_포항시북구_송라면,0
2687,기계면,포항시북구,1,포항시북구,경상북도,포항시북구,경상북도_포항시북구_기계면,0


In [554]:
df_result_1.index

Index([2683, 2684, 2685, 2686, 2687, 2688, 2689, 2690, 2691, 2692,
       ...
       3030, 3031, 3032, 3033, 3034, 3035, 3036, 3037, 3038, 3039],
      dtype='int64', length=462)

In [555]:
len(df_result_1)

462

In [556]:
#행정구를 가지지 않은 경우
print(len(df_result))
df_result_2 = df_result.drop(df_result_1.index)
print(len(df_result_2))

3553
3091


In [557]:
#행정구를 가지는 경우에 대한 처리: 시군구_r에 대해, '시'를 기준으로 분할 후 해당 리스트값을 합쳐주기
print(df_result_1.head())
df_result_1['시군구_r'] = df_result_1['시군구_r'].apply(lambda x : x.split('시')[0] + '시 ' + x.split('시')[1])
print(df_result_1.head())              
df_result_1['시도_시군구_행정동'] = df_result_1['시도'] + '_' + df_result_1['시군구_r'] + '_' + df_result_1['행정동']
print(df_result_1.head())                                    

      행정동    선거구  정수   구시군명    시도  시군구_r      시도_시군구_행정동  cor
2683  흥해읍  포항시북구   1  포항시북구  경상북도  포항시북구  경상북도_포항시북구_흥해읍    0
2684  신광면  포항시북구   1  포항시북구  경상북도  포항시북구  경상북도_포항시북구_신광면    0
2685  청하면  포항시북구   1  포항시북구  경상북도  포항시북구  경상북도_포항시북구_청하면    0
2686  송라면  포항시북구   1  포항시북구  경상북도  포항시북구  경상북도_포항시북구_송라면    0
2687  기계면  포항시북구   1  포항시북구  경상북도  포항시북구  경상북도_포항시북구_기계면    0
      행정동    선거구  정수   구시군명    시도   시군구_r      시도_시군구_행정동  cor
2683  흥해읍  포항시북구   1  포항시북구  경상북도  포항시 북구  경상북도_포항시북구_흥해읍    0
2684  신광면  포항시북구   1  포항시북구  경상북도  포항시 북구  경상북도_포항시북구_신광면    0
2685  청하면  포항시북구   1  포항시북구  경상북도  포항시 북구  경상북도_포항시북구_청하면    0
2686  송라면  포항시북구   1  포항시북구  경상북도  포항시 북구  경상북도_포항시북구_송라면    0
2687  기계면  포항시북구   1  포항시북구  경상북도  포항시 북구  경상북도_포항시북구_기계면    0
      행정동    선거구  정수   구시군명    시도   시군구_r       시도_시군구_행정동  cor
2683  흥해읍  포항시북구   1  포항시북구  경상북도  포항시 북구  경상북도_포항시 북구_흥해읍    0
2684  신광면  포항시북구   1  포항시북구  경상북도  포항시 북구  경상북도_포항시 북구_신광면    0
2685  청하면  포항시북구   1  포항시북구  경상북도  포항시 북구  경상북도_포항시 북구_청하면

In [558]:
#df_result_1 + df_result_2 concat
df_result = pd.concat([df_result_1, df_result_2], axis = 0).sort_index()
df_result.head()

,행정동,선거구,정수,구시군명,시도,시군구_r,시도_시군구_행정동,cor
0,목천읍,천안시갑,1,천안시동남구,충청남도,천안시 동남구,충청남도_천안시 동남구_목천읍,0
1,북면,천안시갑,1,천안시동남구,충청남도,천안시 동남구,충청남도_천안시 동남구_북면,0
2,성남면,천안시갑,1,천안시동남구,충청남도,천안시 동남구,충청남도_천안시 동남구_성남면,0
3,수신면,천안시갑,1,천안시동남구,충청남도,천안시 동남구,충청남도_천안시 동남구_수신면,0
4,병천면,천안시갑,1,천안시동남구,충청남도,천안시 동남구,충청남도_천안시 동남구_병천면,0


In [560]:
#check_point
#제, 행정구에 대한 처리 후 불일치 건수
df_result['행정동'] = df_result['행정동'].apply(lambda x: x.replace('제제','제') if '제제' in x else x)
df_result['시군구_r'] = df_result['시군구_r'].apply(lambda x: '화성시' if '화성' in x else x)
df_result['시도_시군구_행정동'] = df_result['시도'] + '_' + df_result['시군구_r'] + '_' + df_result['행정동']
li_cor = list(gdf['시도_시군구_행정동'])
df_result['cor'] = df_result['시도_시군구_행정동'].\
    apply(lambda x : 1 if x in li_cor\
          else 0)
#'제' 처리 후에는 717건으로 보정
print(len(df_result.loc[df_result['cor'] == 0]))

46


1. 부천시의 경우 : 직접 선거구기입(오정: 갑, 원미: 을, 소사: 정)
2. 합동 및 명칭변경
영종2동 : 24.1 영종동에서 분동 -> 개수영향 x
광주광역시 북구 중흥동 : 24.1 중흥2,3동 합동
안양시 충훈동 : 24.1 석수3동 명칭변경
안양시 관양동 : 24.1 관양1동 명칭변경
안양시 인덕원동 : 24.1 관양2동 명칭변경
예천군 호명읍 : 24.1 호명면에서 승격
울산광역시 복산동 : 24.1 복산1,2동 합동
--------------------------------------
아래 분동 내역은 처리 어려움 but 선거구가 같이 묶어있음
양주시 옥정1,2동 : 24.1 회천4동 분동
오산시 대원1,2동 : 24.1 대원동에서 분동
오산시 신장1,2동 : 24.1 신장동에서 분동
시흥시 거북섬동 : 23.7 정왕2동에서 분동
 
최종레이어 개체 수: 3548 = 3528 - 5(강원군사지역) -2(중흥2+중흥3, 복산1+복산2)+27(부천시)
중선관위 선거구 개체수: 3553 = 3548+분동된 4개 + 나머지 하나는 어디????


In [605]:
gdf.loc[gdf['ADM_NM'].str.contains('복산')]
#merge 후 비어있는 부분에 대한 반영은 qgis에서 
df_result_3 = gdf.merge(df_result[['시도_시군구_행정동','선거구']], left_on = '시도_시군구_행정동', right_on = '시도_시군구_행정동', how = 'left')
df_result_3.columns = ['ADM_NM','ADM_CD','SIGUNGU_NM','SIDO_NM','SIGUNGU_CD','SIDO_CD','geometry','SIDO_SIGUNGU_ADM','cor','DISTRICT']
df_result_3['DIS_DSTRCT'] = df_result_3['SIDO_NM'] + '_' +df_result_3['DISTRICT']
df_result_3.to_file('../shp/행정경계_202402/22대총선_선거구_1', encoding = 'utf-8')


,ADM_NM,ADM_CD,SIGUNGU_NM,SIDO_NM,SIGUNGU_CD,SIDO_CD,geometry
340,복산동,21060520,동래구,부산광역시,21060,21,"POLYGON ((389760.160 291907.297, 389762.806 29..."
654,복산1동,26010540,중구,울산광역시,26010,26,"POLYGON ((410812.754 333012.762, 410823.164 33..."
3013,복산2동,26010550,중구,울산광역시,26010,26,"POLYGON ((411059.635 332770.348, 411066.102 33..."
